# Contruct Prompt

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from us_report_ext import (get_findings, 
                           load_split_md_docs, 
                           get_chroma_retrievers, 
                           retrieve_abnormal_docs)

from us_report_ext._utils import read_markdown

In [2]:
llm_main = ChatOpenAI(model="gpt-3.5-turbo") # Main LLM for Prompt
llm_input = ChatOpenAI(model="gpt-3.5-turbo") # LLM for instruct input
embedding = OpenAIEmbeddings() # Chroma Embedding

## User input to findings

In [3]:
# User Text to Extract
user_text1 = """Generate US report with these findings:
- Mild fatty liver
- 2-mm left renal stone, 5-mm right renal cyst
- A 3-mm gallstone
"""

In [4]:
findings1 = get_findings(input_text=user_text1, llm = llm_input)
findings1

Findings(abnormal_liver=[Liver(finding='Mild fatty liver')], abnormal_kidney=[Kidney(finding='2-mm left renal stone'), Kidney(finding='5-mm right renal cyst')], abnormal_gallbladder=[GallBladder(finding='3-mm gallstone')])

In [5]:
findings1.to_dict()

{'abnormal_liver': ['Mild fatty liver'],
 'abnormal_kidney': ['2-mm left renal stone', '5-mm right renal cyst'],
 'abnormal_gallbladder': ['3-mm gallstone']}

## RAG

### Retrivers

In [6]:
md_header_splits_dict = load_split_md_docs("abnormal")
retriever_dict = get_chroma_retrievers(md_header_splits_dict, embedding= embedding)
retriever_dict

{'abnormal_kidney': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x1194be3b0>, search_kwargs={'k': 3}),
 'abnormal_liver': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x11d4cceb0>, search_kwargs={'k': 3}),
 'abnormal_gallbladder': VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x12d2708e0>, search_kwargs={'k': 3})}

### Retrieve Using `Findings`

In [7]:
abn_doc_dict1 = retrieve_abnormal_docs(retriever_dict, findings1)
abn_doc_dict1

{'abnormal_kidney': [Document(page_content='### Renal Stone  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s).  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s)\n```  \nExamples:  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A few non-obstructing caliceal stones at right kidney.  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- A few non-obstructing caliceal stones at right kidney.\n```', metadata={'Header 1': 'Kidney Findings', 'Header 3': 'Renal Stone'}),
  Document(page_content='# Kidney Findings  \nOrder findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```', 

In [16]:
format_docs(abn_doc_dict1["abnormal_kidney"])

'### Renal Stone  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s).  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s)\n```  \nExamples:  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A few non-obstructing caliceal stones at right kidney.  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- A few non-obstructing caliceal stones at right kidney.\n```\n\n# Kidney Findings  \nOrder findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```\n\n### Renal Cortical Cyst(s)  \nHere is how to report renal cortical cyst according to Bosniak classification system.  \n#### Bosniak 1 (S

In [10]:
[docs.page_content
 for docs in abn_doc_dict1["abnormal_kidney"]]

['### Renal Stone  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s).  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- <quantifier> non-obstructing caliceal stone(s) at `[right | left | both]` kidney(s)\n```  \nExamples:  \n```markdown\n**Kidneys:** Normal size and parenchymal echogenicity of both kidneys. A few non-obstructing caliceal stones at right kidney.  No hydronephrosis or solid mass.\n**IMPRESSION:**\n- A few non-obstructing caliceal stones at right kidney.\n```',
 '# Kidney Findings  \nOrder findings as:\n1. Kidney size and echogenicity\n2. (If any) Renal cyst(s)\n3. (If any) Renal stone, hydronephrosis, or solid mass.  \n```markdown\n**Kidneys:** <kidney_size_echo>. <renal_cyst>. <renal_stone_hydro_solid_mass>.\n```',
 '### Renal Cortical Cyst(s)  \nHere is how to report renal cortical cyst according to Bosniak classification system.  \n#### Bosniak 1

## Prompt

### Function: `get_prompt_template()`

In [56]:
def format_docs(docs):
    if docs == []:
        return ""
    else:
        return "\n\n".join(doc.page_content for doc in docs)

In [57]:
from typing import Dict, List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

def get_prompt_template(abnormal_doc_dict: Dict[str, List[Document]]):
    
    pr_text_intro = read_markdown("prompt/1_introduction.md")
    pr_text_eng = read_markdown("prompt/2_english_style_guide.md")
    pr_text_report_structure = read_markdown("prompt/3_report_structure.md")
    pr_text_temp_normal = read_markdown("prompt/4_report_template_normal.md")
    pr_text_temp_abn = read_markdown("prompt/5_abnormal.md")
    pr_abn_extracted = f"""

    liver:\n{format_docs(abnormal_doc_dict["abnormal_liver"])}

    kidney:\n{format_docs(abnormal_doc_dict["abnormal_kidney"])}

    gallbladder:\n{format_docs(abnormal_doc_dict["abnormal_gallbladder"])}
    """

    # Join them
    pr_text = "\n\n".join([pr_text_intro, pr_text_eng,
                        pr_text_report_structure, pr_text_temp_normal, 
                        pr_text_temp_abn, pr_abn_extracted]) 


    prompt = "\n\n".join([pr_text,
                """
    User input: {user}

    Output:
    """])

    prompt_temp = ChatPromptTemplate.from_template(prompt)
    return prompt_temp
    

In [59]:
get_prompt_template(abn_doc_dict1)

ChatPromptTemplate(input_variables=['user'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user'], template='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.\n\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n- If the user ask "How do I use you?", provide the "User guide", or if not provided, generate it.\n\nYour task: \n- Build radiology report using "Reporting Structure", "Norm

### HowTo: Construct Prompt

In [42]:
pr_text_intro = read_markdown("prompt/1_introduction.md")
pr_text_eng = read_markdown("prompt/2_english_style_guide.md")
pr_text_report_structure = read_markdown("prompt/3_report_structure.md")
pr_text_temp_normal = read_markdown("prompt/4_report_template_normal.md")
pr_text_temp_abn = read_markdown("prompt/5_abnormal.md")

pr_abn_extracted = f"""

liver:\n{format_docs(abn_doc_dict1["abnormal_liver"])}

kidney:\n{format_docs(abn_doc_dict1["abnormal_kidney"])}

gallbladder:\n{format_docs(abn_doc_dict1["abnormal_gallbladder"])}
"""

# Join them
pr_text = "\n\n".join([pr_text_intro, pr_text_eng,
                       pr_text_report_structure, pr_text_temp_normal, 
                       pr_text_temp_abn, pr_abn_extracted]) 


prompt = "\n\n".join([pr_text,
             """
User input: {user}

Output:
"""])

# prompt = "\n\n".join([prompt, f"""
# Input findings: 
# - The following are schema of findings extracted from the user input. if it is empty, consider normal for that organ: 
# {findings1.to_dict()}
# 
# """])

print(prompt)

You are a radiology report writer in my institution. 

I will provide you:

- "English Style Guide" for the preferred ways to write phrases or sentences in the report.
- "Reporting Structure" provides blueprint to build radiology report for each studies.
- "Normal Report Template" provides normal reporting template and normal findings for each studies.
- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.


User role: 
- The user (radiologist) will provide you with ultrasound findings.
- If findings for each specific organ is not provide, assume normal findings for that organ. 
- If the user ask "How do I use you?", provide the "User guide", or if not provided, generate it.

Your task: 
- Build radiology report using "Reporting Structure", "Normal Report Template", and "Abnormal Findings Report Template" with language style according to the "english style guide" as provided.
- Return output as mark

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

prompt_temp = ChatPromptTemplate.from_template(prompt)
prompt_temp

ChatPromptTemplate(input_variables=['user'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user'], template='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.\n\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n- If the user ask "How do I use you?", provide the "User guide", or if not provided, generate it.\n\nYour task: \n- Build radiology report using "Reporting Structure", "Norm

In [44]:
pr_val_1 = prompt_temp.invoke({
            #    "abnormal_gallbladder": "...",
            #    "abnormal_kidney": "...",
            #    "abnormal_liver": "...",
            #   "input_findings": findings1.to_dict(),
               "user": [HumanMessage(content="hi!")]})

pr_val_1

ChatPromptValue(messages=[HumanMessage(content='You are a radiology report writer in my institution. \n\nI will provide you:\n\n- "English Style Guide" for the preferred ways to write phrases or sentences in the report.\n- "Reporting Structure" provides blueprint to build radiology report for each studies.\n- "Normal Report Template" provides normal reporting template and normal findings for each studies.\n- "Abnormal Report Template" provide template to write abnormal findings and corresponding impression for each specific organs and conditions.\n\n\nUser role: \n- The user (radiologist) will provide you with ultrasound findings.\n- If findings for each specific organ is not provide, assume normal findings for that organ. \n- If the user ask "How do I use you?", provide the "User guide", or if not provided, generate it.\n\nYour task: \n- Build radiology report using "Reporting Structure", "Normal Report Template", and "Abnormal Findings Report Template" with language style according t